In [38]:
!pip install PyPDF2==3.0.1

6501.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [39]:
!pip install pdfplumber==0.10.4

6515.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [40]:
!pip install numba==0.59.0

6523.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [41]:
!pip install langchain==0.1.9

6530.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [42]:
!python -m pip install sentence_transformers==2.4.0
#!pip install protobuf

6538.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [43]:
#https://medium.com/@yashashm77/querying-a-pdf-file-using-llm-models-and-sentence-transformer-b3d4d0b40f7d

from PyPDF2 import PdfReader 
from pdfplumber import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import time


from numba import jit, cuda
import numpy as np  

In [46]:
pdf_loader = PdfReader("The Road Not Taken Poem.pdf")
pdf_text = ""
for page_num in range(len(pdf_loader.pages)):
    pdf_page = pdf_loader.pages[page_num]
    pdf_text += pdf_page.extract_text()

pdf_text

'Copyright (c) Houghton Mifflin Company. All Rights Reserved. Two roads diverged in a yellow wood,\nAnd sorry I could not travel bothAnd be one traveler, long I stoodAnd looked down one as far as I couldTo where it bent in the undergrowth;\nThen took the other, as just as fair,\nAnd having perhaps the better claim,Because it was grassy and wanted wear;Though as for that the passing thereHad worn them really about the same,\nAnd both that morning equally lay\nIn leaves no step had trodden black.Oh, I kept the first for another day!Yet knowing how way leads on to way,I doubted if I should ever come back.\nI shall be telling this with a sigh\nSomewhere ages and ages hence:Two roads diverged in a wood, and I—I took the one less traveled by,And that has made all the difference.\n—-Robert Frost—- \nEducation Place: http://www.eduplace.comThe Road Not Taken '

In [47]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=64
    )
split_texts = text_splitter.split_text(pdf_text)
paragraphs = []
for text in split_texts:
    paragraphs.extend(text.split('\n')) 

paragraphs

['Copyright (c) Houghton Mifflin Company. All Rights Reserved. Two roads diverged in a yellow wood,',
 'And sorry I could not travel bothAnd be one traveler, long I stoodAnd looked down one as far as I couldTo where it bent in the undergrowth;',
 'Then took the other, as just as fair,',
 'And having perhaps the better claim,Because it was grassy and wanted wear;Though as for that the passing thereHad worn them really about the same,',
 'And both that morning equally lay',
 'In leaves no step had trodden black.Oh, I kept the first for another day!Yet knowing how way leads on to way,I doubted if I should ever come back.',
 'I shall be telling this with a sigh',
 'Somewhere ages and ages hence:Two roads diverged in a wood, and I—I took the one less traveled by,And that has made all the difference.',
 '—-Robert Frost—- ',
 'Education Place: http://www.eduplace.comThe Road Not Taken']

In [48]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = SentenceTransformer(model_name)

embeddings

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [49]:
responses = []
for paragraph in paragraphs:
        response = embeddings.encode(paragraph, convert_to_tensor=False)
        responses.append((paragraph, response))
responses

[('Copyright (c) Houghton Mifflin Company. All Rights Reserved. Two roads diverged in a yellow wood,',
  array([-3.61503586e-02, -1.56585313e-02,  3.65325399e-02,  1.06952023e-02,
          9.67223272e-02, -1.48610165e-02, -2.79360842e-02,  1.21125774e-02,
          1.62827428e-02, -1.85647383e-02, -1.06522599e-02,  4.84681614e-02,
         -1.51155740e-02,  4.45756316e-02, -1.28444567e-01,  1.46654528e-02,
         -1.14811331e-01,  8.15470591e-02, -3.03981714e-02,  1.42195579e-02,
         -2.47753435e-03,  5.42870387e-02, -3.21062207e-02, -9.41877253e-03,
         -4.12310939e-03,  2.66308635e-02, -3.38393115e-02,  1.01529233e-01,
          5.62929884e-02, -7.38656670e-02, -1.01023698e-02,  1.42790787e-02,
         -8.82179663e-03, -2.19529308e-02,  4.30472456e-02,  2.44150590e-02,
          4.17612568e-02, -7.71362260e-02,  7.54681751e-02, -2.14571618e-02,
         -5.27563244e-02, -1.99110918e-02,  4.58079576e-02,  7.52014220e-02,
         -4.42247912e-02,  4.78214324e-02, -1.4373

In [50]:
temp_list = []
for idx, i in enumerate(responses):
    chunk_name, embedding_array = responses[idx]
    list_embedding_array = embedding_array.tolist()
    temp_list.append({"chunk_name":chunk_name,"embedding_array":list_embedding_array})
temp_list   

[{'chunk_name': 'Copyright (c) Houghton Mifflin Company. All Rights Reserved. Two roads diverged in a yellow wood,',
  'embedding_array': [-0.036150358617305756,
   -0.015658531337976456,
   0.03653253987431526,
   0.010695202276110649,
   0.0967223271727562,
   -0.014861016534268856,
   -0.02793608419597149,
   0.012112577445805073,
   0.016282742843031883,
   -0.01856473833322525,
   -0.010652259923517704,
   0.04846816137433052,
   -0.01511557400226593,
   0.044575631618499756,
   -0.12844456732273102,
   0.014665452763438225,
   -0.11481133103370667,
   0.08154705911874771,
   -0.0303981713950634,
   0.014219557866454124,
   -0.0024775343481451273,
   0.05428703874349594,
   -0.032106220722198486,
   -0.009418772533535957,
   -0.004123109392821789,
   0.026630863547325134,
   -0.03383931145071983,
   0.10152923315763474,
   0.05629298835992813,
   -0.07386566698551178,
   -0.010102369822561741,
   0.0142790786921978,
   -0.008821796625852585,
   -0.021952930837869644,
   0.04304724

In [51]:
#examples of how 1 pair of chunk_name and embedding_array generated
responses[0]

chunk_name, embedding_array = responses[0]
chunk_name
embedding_array

list_embedding_array = embedding_array.tolist()
list_embedding_array
result = {"chunk_name":chunk_name,"embedding":list_embedding_array}
result

{'chunk_name': 'Copyright (c) Houghton Mifflin Company. All Rights Reserved. Two roads diverged in a yellow wood,',
 'embedding': [-0.036150358617305756,
  -0.015658531337976456,
  0.03653253987431526,
  0.010695202276110649,
  0.0967223271727562,
  -0.014861016534268856,
  -0.02793608419597149,
  0.012112577445805073,
  0.016282742843031883,
  -0.01856473833322525,
  -0.010652259923517704,
  0.04846816137433052,
  -0.01511557400226593,
  0.044575631618499756,
  -0.12844456732273102,
  0.014665452763438225,
  -0.11481133103370667,
  0.08154705911874771,
  -0.0303981713950634,
  0.014219557866454124,
  -0.0024775343481451273,
  0.05428703874349594,
  -0.032106220722198486,
  -0.009418772533535957,
  -0.004123109392821789,
  0.026630863547325134,
  -0.03383931145071983,
  0.10152923315763474,
  0.05629298835992813,
  -0.07386566698551178,
  -0.010102369822561741,
  0.0142790786921978,
  -0.008821796625852585,
  -0.021952930837869644,
  0.04304724559187889,
  0.02441505901515484,
  0.0417

In [52]:
import requests
import json

In [53]:
payload2 = json.dumps({
    "collection":"Test",
    "database":"Test",
    "dataSource":"Cluster0",
    "documents": temp_list #put in list
})

headers = {
  'Content-Type': 'application/json',
  'Access-Control-Request-Headers': '*',
  'api-key': 'Phu3o72HM9itPsmX6e1Tb4S0aAuwyz8BrFmWPalg9HJs9vCRZ45YktglrKnrvIxg',
}

url_insert = "https://us-east-2.aws.data.mongodb-api.com/app/data-bvuiv/endpoint/data/v1/action/insertMany"

#response_2 = requests.post(url_insert, headers=headers, json=data_1)
response_2 = requests.request("POST", url_insert, headers=headers, data=payload2)
print(response_2.text)

{"insertedIds":["6601f272268a5ecef6e56674","6601f272268a5ecef6e56675","6601f272268a5ecef6e56676","6601f272268a5ecef6e56677","6601f272268a5ecef6e56678","6601f272268a5ecef6e56679","6601f272268a5ecef6e5667a","6601f272268a5ecef6e5667b","6601f272268a5ecef6e5667c","6601f272268a5ecef6e5667d"]}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=49d00240-a8a6-4b0d-8544-487265eb2619' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>